In [1]:
# install from PyPI
!pip install openai

In [2]:
# OpenAI kütüphanesini yükle
!pip install openai --quiet


1. Adım: Proje klasör yapısı oluşturma

In [45]:
import os

# Colab için klasör yapısını oluştur
os.makedirs("genai_test_generator/src", exist_ok=True)
os.makedirs("genai_test_generator/tests", exist_ok=True)


2. Adım: Test Edilecek Python Fonksiyonları Yazma

In [80]:
# math_utils.py dosyasını oluştur
math_utils_code = """
def add(a, b):
    \"\"\"Returns the sum of a and b\"\"\"
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("add() only accepts numbers")
    return a + b

def divide(a, b):
    \"\"\"Returns the result of dividing a by b\"\"\"
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

def factorial(n):
    \"\"\"Returns factorial of n\"\"\"
    if n < 0:
        raise ValueError("Negative number not allowed")
    if n == 0:
        return 1
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result
"""

with open("genai_test_generator/src/math_utils.py", "w", encoding="utf-8") as f:
    f.write(math_utils_code)

print("✅ math_utils.py dosyası oluşturuldu.")


✅ math_utils.py dosyası oluşturuldu.


3. OpenAI ile Otomatik Test Üret (LLM)

In [5]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.91.0
    Uninstalling openai-1.91.0:
      Successfully uninstalled openai-1.91.0


In [ ]:
import openai
import os
import re

# OpenAI API anahtarını girin
openai.api_key = ""

# Dosya yolları
source_file_path = "genai_test_generator/src/math_utils.py"
output_dir = "genai_test_generator/tests"
os.makedirs(output_dir, exist_ok=True)

# Fonksiyon kodunu kaynak dosyadan çıkar
def extract_function(source_code, function_name):
    lines = source_code.splitlines()
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith(f"def {function_name}("):
            start_index = i
            break
    if start_index is None:
        raise ValueError(f"{function_name} fonksiyonu bulunamadı.")
    fn_lines = [lines[start_index]]
    indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    for line in lines[start_index + 1:]:
        if len(line.strip()) == 0 or (len(line) - len(line.lstrip())) > indent:
            fn_lines.append(line)
        else:
            break
    return "\n".join(fn_lines)

# LLM ile test üret
def generate_test_code(function_code):
    prompt = f"""Aşağıdaki Python fonksiyonu için pytest kullanarak test fonksiyonları yaz. Sadece Python test kodu ver. Açıklama, yorum veya metin yazma:\n```python\n{function_code}\n```"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

# LLM çıktısını temizle
def clean_test_code(llm_output):
    # 1. Kod bloğunu izole et (```python ... ```)
    code_blocks = re.findall(r"```(?:python)?(.*?)```", llm_output, re.DOTALL)
    cleaned = code_blocks[0].strip() if code_blocks else llm_output.strip()

    # 2. Tüm 'from <herhangi_bir_modül> import ...' satırlarını tespit et ve düzelt
    cleaned = re.sub(
        r"from\s+[\w_\.]+?\s+import",  # örn: from mymodule import
        "from genai_test_generator.src.math_utils import",
        cleaned
    )

    return cleaned

# Kaynak dosyayı oku
with open(source_file_path, "r", encoding="utf-8") as f:
    source_code = f.read()

# Fonksiyonlar için test üret
functions = ["add", "divide", "factorial"]

for fn in functions:
    try:
        fn_code = extract_function(source_code, fn)
        raw_output = generate_test_code(fn_code)
        test_code = clean_test_code(raw_output)

        test_path = os.path.join(output_dir, f"test_{fn}_llm.py")
        with open(test_path, "w", encoding="utf-8") as f_out:
            f_out.write(test_code)

        print(f"✅ {fn} fonksiyonu için test dosyası oluşturuldu: {test_path}")
    except Exception as e:
        print(f"❌ {fn} fonksiyonu için test oluşturulurken hata oluştu: {e}")


✅ add fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_add_llm.py
✅ divide fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_divide_llm.py
✅ factorial fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_factorial_llm.py


4. Oluşan Test Dosyalarını Göster

In [7]:
# Test dosyalarının içeriğini görüntüle
for fn in functions:
    test_path = f"genai_test_generator/tests/test_{fn}_llm.py"
    print(f"\n--- {test_path} ---\n")
    with open(test_path, "r", encoding="utf-8") as f:
        print(f.read())



--- genai_test_generator/tests/test_add_llm.py ---

import pytest
from genai_test_generator.src.math_utils import add  # assuming the function is in a file called my_module.py

def test_add():
    assert add(1, 2) == 3
    assert add(-1, 1) == 0
    assert add(0, 0) == 0

def test_add_floats():
    assert add(1.5, 2.5) == 4.0
    assert add(-1.5, 1.5) == 0.0

def test_add_strings():
    assert add("hello", " world") == "hello world"
    with pytest.raises(TypeError):
        add("hello", 3)

--- genai_test_generator/tests/test_divide_llm.py ---

import pytest
from genai_test_generator.src.math_utils import divide

def test_divide():
    assert divide(10, 2) == 5
    assert divide(15, 3) == 5
    assert divide(-4, 2) == -2
    assert divide(0, 1) == 0

def test_divide_by_zero():
    with pytest.raises(ValueError) as e:
        divide(10, 0)
    assert str(e.value) == "Cannot divide by zero"

--- genai_test_generator/tests/test_factorial_llm.py ---

import pytest
from genai_test_generat

In [48]:
# Gerekli test kütüphanelerini yükleyelim
!pip install pytest coverage --quiet


 6. Adım: Dosya Yapısını Colab İçin Uygun Hale Getirme

In [9]:
# __init__.py dosyasını ekle (boş dosya)
open("genai_test_generator/__init__.py", "w").close()
open("genai_test_generator/src/__init__.py", "w").close()
open("genai_test_generator/tests/__init__.py", "w").close()


7. Adım: Testleri pytest ile çalıştır

In [100]:
# pytest ile testleri çalıştır
!pytest genai_test_generator/tests/ --disable-warnings


============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.3.5, pluggy-1.6.0
rootdir: /content
plugins: cov-6.2.1, anyio-4.9.0, typeguard-4.4.4, langsmith-0.4.1
collected 12 items                                                             

genai_test_generator/tests/test_add_numbers_llm.py ..                    [ 16%]
genai_test_generator/tests/test_check_password_strength_llm.py F.        [ 33%]
genai_test_generator/tests/test_compute_factorial_llm.py ..              [ 50%]
genai_test_generator/tests/test_divide_numbers_llm.py ..                 [ 66%]
genai_test_generator/tests/test_is_palindrome_llm.py ...                 [ 91%]
genai_test_generator/tests/test_summarize_statistics_llm.py .            [100%]

=================================== FAILURES ===================================
_________________________ test_check_password_strength _________________________

    def test_check_password_strength():
        a

8. Adım: Coverage (Kod Kapsama) Raporu Al

In [101]:
# Coverage ile kapsama analizi yap
!coverage run -m pytest genai_test_generator/tests/ --disable-warnings
!coverage report -m


============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.3.5, pluggy-1.6.0
rootdir: /content
plugins: cov-6.2.1, anyio-4.9.0, typeguard-4.4.4, langsmith-0.4.1
collected 12 items                                                             

genai_test_generator/tests/test_add_numbers_llm.py ..                    [ 16%]
genai_test_generator/tests/test_check_password_strength_llm.py F.        [ 33%]
genai_test_generator/tests/test_compute_factorial_llm.py ..              [ 50%]
genai_test_generator/tests/test_divide_numbers_llm.py ..                 [ 66%]
genai_test_generator/tests/test_is_palindrome_llm.py ...                 [ 91%]
genai_test_generator/tests/test_summarize_statistics_llm.py .            [100%]

=================================== FAILURES ===================================
_________________________ test_check_password_strength _________________________

    def test_check_password_strength():
        a

Code Coverage detay çalışmaları :

In [102]:
!pip install pytest pytest-cov


In [103]:
!pytest --cov=genai_test_generator.src --cov-report=term-missing


============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.3.5, pluggy-1.6.0
rootdir: /content
plugins: cov-6.2.1, anyio-4.9.0, typeguard-4.4.4, langsmith-0.4.1
collected 12 items                                                             

genai_test_generator/tests/test_add_numbers_llm.py ..                    [ 16%]
genai_test_generator/tests/test_check_password_strength_llm.py F.        [ 33%]
genai_test_generator/tests/test_compute_factorial_llm.py ..              [ 50%]
genai_test_generator/tests/test_divide_numbers_llm.py ..                 [ 66%]
genai_test_generator/tests/test_is_palindrome_llm.py ...                 [ 91%]
genai_test_generator/tests/test_summarize_statistics_llm.py .            [100%]

=================================== FAILURES ===================================
_________________________ test_check_password_strength _________________________

    def test_check_password_strength():
        a

In [104]:
!pytest --cov=genai_test_generator.src.math_utils --cov-report=term-missing


============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.3.5, pluggy-1.6.0
rootdir: /content
plugins: cov-6.2.1, anyio-4.9.0, typeguard-4.4.4, langsmith-0.4.1
collected 12 items                                                             

genai_test_generator/tests/test_add_numbers_llm.py ..                    [ 16%]
genai_test_generator/tests/test_check_password_strength_llm.py F.        [ 33%]
genai_test_generator/tests/test_compute_factorial_llm.py ..              [ 50%]
genai_test_generator/tests/test_divide_numbers_llm.py ..                 [ 66%]
genai_test_generator/tests/test_is_palindrome_llm.py ...                 [ 91%]
genai_test_generator/tests/test_summarize_statistics_llm.py .            [100%]

=================================== FAILURES ===================================
_________________________ test_check_password_strength _________________________

    def test_check_password_strength():
        a

 9. Adım: HTML Formatında Coverage Raporu

In [92]:
# HTML formatında coverage raporu oluştur
!coverage html

# HTML dosyalarını tarayıcıda gösterme işlemi sadece yerel ortamda işe yarar.
# Ancak çıktı dosyalarını zipleyip indirebiliriz:
import shutil

shutil.make_archive("coverage_html_report", 'zip', "htmlcov")


Wrote HTML report to ]8;;file:///content/htmlcov/index.htmlhtmlcov/index.html]8;;


'/content/coverage_html_report.zip'

10. Adım : Sonuçları Tablolaştırma

In [28]:
import pandas as pd

data = {
    "Fonksiyon": ["add", "divide", "factorial"],
    "Test Tipi": ["LLM"] * 3,
    "Kod Kapsamı (%)": [100, 100, 100],  # Gerçek coverage sonuçlarına göre güncellendi
    "Test Sayısı": [2, 2, 4],            # pytest ile geçen test sayısı
    "Edge Case Durumu": ["✅", "✅", "✅"],  # Tüm kritik durumlar test edildi
    "Üretim Süresi (sn)": [4.2, 4.5, 4.1], # Tahmini LLM cevap süreleri
}

df = pd.DataFrame(data)
print(df)


   Fonksiyon Test Tipi  Kod Kapsamı (%)  Test Sayısı Edge Case Durumu  \
0        add       LLM              100            2                ✅   
1     divide       LLM              100            2                ✅   
2  factorial       LLM              100            4                ✅   

   Üretim Süresi (sn)  
0                 4.2  
1                 4.5  
2                 4.1  


Projeyi ZIP Olarak İndirme

In [29]:
import shutil

shutil.make_archive("genai_test_project", 'zip', "genai_test_generator")


'/content/genai_test_project.zip'

Çalışan ve sonuçları tablolaştıran Çözüm :

In [ ]:
import openai
import os
import re
import subprocess
import time
import pandas as pd

# OpenAI API anahtarını girin
openai.api_key = ""  # kendi anahtarını yaz

source_file_path = "genai_test_generator/src/math_utils.py"
output_dir = "genai_test_generator/tests"
os.makedirs(output_dir, exist_ok=True)

def extract_function(source_code, function_name):
    lines = source_code.splitlines()
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith(f"def {function_name}("):
            start_index = i
            break
    if start_index is None:
        raise ValueError(f"{function_name} fonksiyonu bulunamadı.")
    fn_lines = [lines[start_index]]
    indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    for line in lines[start_index + 1:]:
        if len(line.strip()) == 0 or (len(line) - len(line.lstrip())) > indent:
            fn_lines.append(line)
        else:
            break
    return "\n".join(fn_lines)

def generate_test_code(function_code):
    prompt = f"""Aşağıdaki Python fonksiyonu için pytest kullanarak test fonksiyonları yaz. Sadece Python test kodu ver. Açıklama, yorum veya metin yazma:\n```python\n{function_code}\n```"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

def clean_test_code(llm_output):
    code_blocks = re.findall(r"```(?:python)?(.*?)```", llm_output, re.DOTALL)
    cleaned = code_blocks[0].strip() if code_blocks else llm_output.strip()
    cleaned = re.sub(r"from\s+[\w_\.]+?\s+import", "from genai_test_generator.src.math_utils import", cleaned)
    return cleaned

def get_test_count(test_path):
    result = subprocess.run(["pytest", test_path, "-q"], capture_output=True, text=True)
    output = result.stdout
    match = re.search(r"collected\s+(\d+)\s+items", output)
    if match:
        return int(match.group(1))
    return 0

def get_coverage():
    result = subprocess.run(["pytest", "--cov=genai_test_generator/src", "--cov-report=term"], capture_output=True, text=True)
    lines = result.stdout.splitlines()
    for line in lines:
        if "math_utils.py" in line:
            parts = line.split()
            try:
                return int(parts[3].replace("%", ""))
            except:
                pass
    return 0

source_code = open(source_file_path, "r", encoding="utf-8").read()

functions = ["add", "divide", "factorial"]

data = {
    "Fonksiyon": [],
    "Test Tipi": [],
    "Kod Kapsamı (%)": [],
    "Test Sayısı": [],
    "Edge Case Durumu": [],
    "Üretim Süresi (sn)": [],
}

for fn in functions:
    try:
        fn_code = extract_function(source_code, fn)

        start_time = time.time()
        raw_output = generate_test_code(fn_code)
        elapsed = time.time() - start_time

        test_code = clean_test_code(raw_output)

        test_path = os.path.join(output_dir, f"test_{fn}_llm.py")
        with open(test_path, "w", encoding="utf-8") as f_out:
            f_out.write(test_code)

        test_count = get_test_count(test_path)
        coverage = get_coverage()
        edge_case = "✅"  # İstersen daha detaylı kontrol ekleyebilirsin

        data["Fonksiyon"].append(fn)
        data["Test Tipi"].append("LLM")
        data["Kod Kapsamı (%)"].append(coverage if coverage is not None else 0)
        data["Test Sayısı"].append(test_count)
        data["Edge Case Durumu"].append(edge_case)
        data["Üretim Süresi (sn)"].append(round(elapsed, 2))

        print(f"✅ {fn} fonksiyonu için test dosyası oluşturuldu: {test_path}")
    except Exception as e:
        print(f"❌ {fn} fonksiyonu için test oluşturulurken hata oluştu: {e}")

df = pd.DataFrame(data)
print(df)


✅ add fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_add_llm.py
✅ divide fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_divide_llm.py
✅ factorial fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_factorial_llm.py
   Fonksiyon Test Tipi  Kod Kapsamı (%)  Test Sayısı Edge Case Durumu  \
0        add       LLM               27            0                ✅   
1     divide       LLM               47            0                ✅   
2  factorial       LLM              100            0                ✅   

   Üretim Süresi (sn)  
0                5.94  
1                6.84  
2                7.55  


Birim testleri EDGE CASE yöntemi ile üretiyor ve sonuçlarını alıyor:

In [ ]:
import openai
import os
import re
import subprocess
import time
import pandas as pd

# OpenAI API anahtarını girin
openai.api_key = ""  # kendi anahtarını yaz

source_file_path = "genai_test_generator/src/math_utils.py"
output_dir = "genai_test_generator/tests"
os.makedirs(output_dir, exist_ok=True)

def extract_function(source_code, function_name):
    lines = source_code.splitlines()
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith(f"def {function_name}("):
            start_index = i
            break
    if start_index is None:
        raise ValueError(f"{function_name} fonksiyonu bulunamadı.")
    fn_lines = [lines[start_index]]
    indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    for line in lines[start_index + 1:]:
        if len(line.strip()) == 0 or (len(line) - len(line.lstrip())) > indent:
            fn_lines.append(line)
        else:
            break
    return "\n".join(fn_lines)

def generate_test_code(function_code):
   #  prompt = f"""Aşağıdaki Python fonksiyonu için pytest kullanarak test fonksiyonları yaz.
   #  Sadece Python test kodu ver. Açıklama, yorum veya metin yazma:\n```python\n{function_code}\n```"""
    prompt = f"""
    Aşağıdaki Python fonksiyonu için pytest kullanarak test kodu üret.
    - Normal durumları test et
    - Edge-case senaryolarını da mutlaka dahil et (örneğin negatif sayılar, sıfır, yanlış veri tipi, boş giriş).
    Sadece Python test kodunu ver. Açıklama yazma.\n```python\n{function_code}\n```"""


    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

def clean_test_code(llm_output):
    code_blocks = re.findall(r"```(?:python)?(.*?)```", llm_output, re.DOTALL)
    cleaned = code_blocks[0].strip() if code_blocks else llm_output.strip()
    cleaned = re.sub(r"from\s+[\w_\.]+?\s+import", "from genai_test_generator.src.math_utils import", cleaned)
    return cleaned

def get_test_count(test_path):
    result = subprocess.run(["pytest", test_path, "-q"], capture_output=True, text=True)
    output = result.stdout
    match = re.search(r"collected\s+(\d+)\s+items", output)
    if match:
        return int(match.group(1))
    return 0

def get_coverage():
    result = subprocess.run(["pytest", "--cov=genai_test_generator/src", "--cov-report=term"], capture_output=True, text=True)
    lines = result.stdout.splitlines()
    for line in lines:
        if "math_utils.py" in line:
            parts = line.split()
            try:
                return int(parts[3].replace("%", ""))
            except:
                pass
    return 0

source_code = open(source_file_path, "r", encoding="utf-8").read()

functions = ["add", "divide", "factorial"]

data = {
    "Fonksiyon": [],
    "Test Tipi": [],
    "Kod Kapsamı (%)": [],
    "Test Sayısı": [],
    "Edge Case Durumu": [],
    "Üretim Süresi (sn)": [],
}

for fn in functions:
    try:
        fn_code = extract_function(source_code, fn)

        start_time = time.time()
        raw_output = generate_test_code(fn_code)
        elapsed = time.time() - start_time

        test_code = clean_test_code(raw_output)

        test_path = os.path.join(output_dir, f"test_{fn}_llm.py")
        with open(test_path, "w", encoding="utf-8") as f_out:
            f_out.write(test_code)

        test_count = get_test_count(test_path)
        coverage = get_coverage()
        edge_case = "✅"  # İstersen daha detaylı kontrol ekleyebilirsin

        data["Fonksiyon"].append(fn)
        data["Test Tipi"].append("LLM")
        data["Kod Kapsamı (%)"].append(coverage if coverage is not None else 0)
        data["Test Sayısı"].append(test_count)
        data["Edge Case Durumu"].append(edge_case)
        data["Üretim Süresi (sn)"].append(round(elapsed, 2))

        print(f"✅ {fn} fonksiyonu için test dosyası oluşturuldu: {test_path}")
    except Exception as e:
        print(f"❌ {fn} fonksiyonu için test oluşturulurken hata oluştu: {e}")

df = pd.DataFrame(data)
print(df)


✅ add fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_add_llm.py
✅ divide fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_divide_llm.py
✅ factorial fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_factorial_llm.py
   Fonksiyon Test Tipi  Kod Kapsamı (%)  Test Sayısı Edge Case Durumu  \
0        add       LLM              100            0                ✅   
1     divide       LLM              100            0                ✅   
2  factorial       LLM              100            0                ✅   

   Üretim Süresi (sn)  
0                5.09  
1                8.21  
2                5.94  


In [20]:
# index.html içeriğini Colab'da görüntülemek
from IPython.core.display import display, HTML

with open("htmlcov/index.html", "r", encoding="utf-8") as f:
    html_content = f.read()

display(HTML(html_content))


File,statements,missing,excluded,coverage
genai_test_generator/src/math_utils.py,15,0,0,100%
Total,15,0,0,100%


**gpt-4o Kullanımı**

In [ ]:
import openai
import os
import re
import subprocess
import time
import pandas as pd

# OpenAI API anahtarını girin
openai.api_key = ""  # kendi anahtarını yaz

source_file_path = "genai_test_generator/src/math_utils.py"
output_dir = "genai_test_generator/tests"
os.makedirs(output_dir, exist_ok=True)

def extract_function(source_code, function_name):
    lines = source_code.splitlines()
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith(f"def {function_name}("):
            start_index = i
            break
    if start_index is None:
        raise ValueError(f"{function_name} fonksiyonu bulunamadı.")
    fn_lines = [lines[start_index]]
    indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    for line in lines[start_index + 1:]:
        if len(line.strip()) == 0 or (len(line) - len(line.lstrip())) > indent:
            fn_lines.append(line)
        else:
            break
    return "\n".join(fn_lines)

def generate_test_code(function_code):
   #  prompt = f"""Aşağıdaki Python fonksiyonu için pytest kullanarak test fonksiyonları yaz.
   #  Sadece Python test kodu ver. Açıklama, yorum veya metin yazma:\n```python\n{function_code}\n```"""
    prompt = f"""
    Aşağıdaki Python fonksiyonu için pytest kullanarak test kodu üret.
    - Normal durumları test et
    - Edge-case senaryolarını da mutlaka dahil et (örneğin negatif sayılar, sıfır, yanlış veri tipi, boş giriş).
    Sadece Python test kodunu ver. Açıklama yazma.\n```python\n{function_code}\n```"""


    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

def clean_test_code(llm_output, function_name):
    # Kod bloklarını ayıkla (```python ... ```)
    code_blocks = re.findall(r"```(?:python)?(.*?)```", llm_output, re.DOTALL)
    cleaned = code_blocks[0].strip() if code_blocks else llm_output.strip()

    lines = cleaned.splitlines()
    new_lines = []
    import_found = False
    pytest_import_found = False

    for line in lines:
        stripped = line.strip()

        # pytest import varsa işaretle
        if stripped.startswith("import pytest"):
            pytest_import_found = True
            new_lines.append(line)
            continue

        # doğru import satırı varsa işaretle
        if (
            f"from genai_test_generator.src.math_utils import {function_name}" in stripped
            or re.match(rf"from\s+\S+\s+import\s+{function_name}", stripped)
        ):
            import_found = True
            continue  # tüm importları kaldırıp bizim ekleyeceğimiz için bunu atla

        # gereksiz diğer importları atla
        if stripped.startswith("import ") or stripped.startswith("from "):
            continue

        new_lines.append(line)

    # En başa pytest import'u yoksa ekle
    if not pytest_import_found:
        new_lines.insert(0, "import pytest")

    # pytest'ten sonra doğru math_utils import satırını ekle
    for i, line in enumerate(new_lines):
        if line.strip().startswith("import pytest"):
            new_lines.insert(i + 1, f"from genai_test_generator.src.math_utils import {function_name}")
            break
    else:
        new_lines.insert(0, f"from genai_test_generator.src.math_utils import {function_name}")

    return "\n".join(new_lines)





def get_test_count(test_path):
    result = subprocess.run(["pytest", test_path, "-q"], capture_output=True, text=True)
    output = result.stdout
    match = re.search(r"collected\s+(\d+)\s+items", output)
    if match:
        return int(match.group(1))
    return 0

def get_coverage():
    result = subprocess.run(["pytest", "--cov=genai_test_generator/src", "--cov-report=term"], capture_output=True, text=True)
    lines = result.stdout.splitlines()
    for line in lines:
        if "math_utils.py" in line:
            parts = line.split()
            try:
                return int(parts[3].replace("%", ""))
            except:
                pass
    return 0

source_code = open(source_file_path, "r", encoding="utf-8").read()

functions = ["add", "divide", "factorial"]

data = {
    "Fonksiyon": [],
    "Test Tipi": [],
    "Kod Kapsamı (%)": [],
    "Test Sayısı": [],
    "Edge Case Durumu": [],
    "Üretim Süresi (sn)": [],
}

for fn in functions:
    try:
        fn_code = extract_function(source_code, fn)

        start_time = time.time()
        raw_output = generate_test_code(fn_code)
        elapsed = time.time() - start_time

        test_code = clean_test_code(raw_output, fn)

        test_path = os.path.join(output_dir, f"test_{fn}_llm.py")
        with open(test_path, "w", encoding="utf-8") as f_out:
            f_out.write(test_code)

        test_count = get_test_count(test_path)
        coverage = get_coverage()
        edge_case = "✅"  # İstersen daha detaylı kontrol ekleyebilirsin

        data["Fonksiyon"].append(fn)
        data["Test Tipi"].append("LLM")
        data["Kod Kapsamı (%)"].append(coverage if coverage is not None else 0)
        data["Test Sayısı"].append(test_count)
        data["Edge Case Durumu"].append(edge_case)
        data["Üretim Süresi (sn)"].append(round(elapsed, 2))

        print(f"✅ {fn} fonksiyonu için test dosyası oluşturuldu: {test_path}")
    except Exception as e:
        print(f"❌ {fn} fonksiyonu için test oluşturulurken hata oluştu: {e}")

df = pd.DataFrame(data)
print(df)


✅ add fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_add_llm.py
✅ divide fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_divide_llm.py
✅ factorial fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_factorial_llm.py
   Fonksiyon Test Tipi  Kod Kapsamı (%)  Test Sayısı Edge Case Durumu  \
0        add       LLM              100            0                ✅   
1     divide       LLM              100            0                ✅   
2  factorial       LLM              100            0                ✅   

   Üretim Süresi (sn)  
0                3.63  
1                3.17  
2                1.51  


**gpt-3.5-turbo** kullanımı

In [ ]:
import openai
import os
import re
import subprocess
import time
import pandas as pd

# OpenAI API anahtarını girin
openai.api_key = ""  # kendi anahtarını yaz

source_file_path = "genai_test_generator/src/math_utils.py"
output_dir = "genai_test_generator/tests"
os.makedirs(output_dir, exist_ok=True)

def extract_function(source_code, function_name):
    lines = source_code.splitlines()
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith(f"def {function_name}("):
            start_index = i
            break
    if start_index is None:
        raise ValueError(f"{function_name} fonksiyonu bulunamadı.")
    fn_lines = [lines[start_index]]
    indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    for line in lines[start_index + 1:]:
        if len(line.strip()) == 0 or (len(line) - len(line.lstrip())) > indent:
            fn_lines.append(line)
        else:
            break
    return "\n".join(fn_lines)

def generate_test_code(function_code):
   #  prompt = f"""Aşağıdaki Python fonksiyonu için pytest kullanarak test fonksiyonları yaz.
   #  Sadece Python test kodu ver. Açıklama, yorum veya metin yazma:\n```python\n{function_code}\n```"""
    prompt = f"""
    Aşağıdaki Python fonksiyonu için pytest kullanarak test kodu üret.
    - Normal durumları test et
    - Edge-case senaryolarını da mutlaka dahil et (örneğin negatif sayılar, sıfır, yanlış veri tipi, boş giriş).
    Sadece Python test kodunu ver. Açıklama yazma.\n```python\n{function_code}\n```"""


    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

def clean_test_code(llm_output, function_name):
    # Kod bloklarını ayıkla (```python ... ```)
    code_blocks = re.findall(r"```(?:python)?(.*?)```", llm_output, re.DOTALL)
    cleaned = code_blocks[0].strip() if code_blocks else llm_output.strip()

    lines = cleaned.splitlines()
    new_lines = []
    import_found = False
    pytest_import_found = False

    for line in lines:
        stripped = line.strip()

        # pytest import varsa işaretle
        if stripped.startswith("import pytest"):
            pytest_import_found = True
            new_lines.append(line)
            continue

        # doğru import satırı varsa işaretle
        if (
            f"from genai_test_generator.src.math_utils import {function_name}" in stripped
            or re.match(rf"from\s+\S+\s+import\s+{function_name}", stripped)
        ):
            import_found = True
            continue  # tüm importları kaldırıp bizim ekleyeceğimiz için bunu atla

        # gereksiz diğer importları atla
        if stripped.startswith("import ") or stripped.startswith("from "):
            continue

        new_lines.append(line)

    # En başa pytest import'u yoksa ekle
    if not pytest_import_found:
        new_lines.insert(0, "import pytest")

    # pytest'ten sonra doğru math_utils import satırını ekle
    for i, line in enumerate(new_lines):
        if line.strip().startswith("import pytest"):
            new_lines.insert(i + 1, f"from genai_test_generator.src.math_utils import {function_name}")
            break
    else:
        new_lines.insert(0, f"from genai_test_generator.src.math_utils import {function_name}")

    return "\n".join(new_lines)





def get_test_count(test_path):
    result = subprocess.run(["pytest", test_path, "-q"], capture_output=True, text=True)
    output = result.stdout
    match = re.search(r"collected\s+(\d+)\s+items", output)
    if match:
        return int(match.group(1))
    return 0

def get_coverage():
    result = subprocess.run(["pytest", "--cov=genai_test_generator/src", "--cov-report=term"], capture_output=True, text=True)
    lines = result.stdout.splitlines()
    for line in lines:
        if "math_utils.py" in line:
            parts = line.split()
            try:
                return int(parts[3].replace("%", ""))
            except:
                pass
    return 0

source_code = open(source_file_path, "r", encoding="utf-8").read()

functions = ["add", "divide", "factorial"]

data = {
    "Fonksiyon": [],
    "Test Tipi": [],
    "Kod Kapsamı (%)": [],
    "Test Sayısı": [],
    "Edge Case Durumu": [],
    "Üretim Süresi (sn)": [],
}

for fn in functions:
    try:
        fn_code = extract_function(source_code, fn)

        start_time = time.time()
        raw_output = generate_test_code(fn_code)
        elapsed = time.time() - start_time

        test_code = clean_test_code(raw_output, fn)

        test_path = os.path.join(output_dir, f"test_{fn}_llm.py")
        with open(test_path, "w", encoding="utf-8") as f_out:
            f_out.write(test_code)

        test_count = get_test_count(test_path)
        coverage = get_coverage()
        edge_case = "✅"  # İstersen daha detaylı kontrol ekleyebilirsin

        data["Fonksiyon"].append(fn)
        data["Test Tipi"].append("LLM")
        data["Kod Kapsamı (%)"].append(coverage if coverage is not None else 0)
        data["Test Sayısı"].append(test_count)
        data["Edge Case Durumu"].append(edge_case)
        data["Üretim Süresi (sn)"].append(round(elapsed, 2))

        print(f"✅ {fn} fonksiyonu için test dosyası oluşturuldu: {test_path}")
    except Exception as e:
        print(f"❌ {fn} fonksiyonu için test oluşturulurken hata oluştu: {e}")

df = pd.DataFrame(data)
print(df)


✅ add fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_add_llm.py
✅ divide fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_divide_llm.py
✅ factorial fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_factorial_llm.py
   Fonksiyon Test Tipi  Kod Kapsamı (%)  Test Sayısı Edge Case Durumu  \
0        add       LLM              100            0                ✅   
1     divide       LLM              100            0                ✅   
2  factorial       LLM              100            0                ✅   

   Üretim Süresi (sn)  
0                4.35  
1                3.14  
2                1.40  


DAHA PROFESYONELCE ÇALIŞALIM :

In [83]:
# math_utils.py
math_utils_code = """
def add_numbers(a: float, b: float) -> float:
    \"\"\"Adds two numeric values and returns the result.\"\"\"
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("Inputs must be numeric types.")
    return a + b

def divide_numbers(numerator: float, denominator: float) -> float:
    \"\"\"Divides the numerator by the denominator and returns the result.\"\"\"
    if not isinstance(numerator, (int, float)) or not isinstance(denominator, (int, float)):
        raise TypeError("Both numerator and denominator must be numeric.")
    if denominator == 0:
        raise ZeroDivisionError("Cannot divide by zero.")
    return numerator / denominator

def compute_factorial(n: int) -> int:
    \"\"\"Computes the factorial of a non-negative integer.\"\"\"
    if not isinstance(n, int):
        raise TypeError("Input must be an integer.")
    if n < 0:
        raise ValueError("Negative values are not allowed.")
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result
"""

with open("genai_test_generator/src/math_utils.py", "w", encoding="utf-8") as f:
    f.write(math_utils_code)

print("✅ math_utils.py dosyası oluşturuldu.")

✅ math_utils.py dosyası oluşturuldu.


In [ ]:
import openai
import os
import re
import subprocess
import time
import pandas as pd

# OpenAI API anahtarını girin
openai.api_key = ""  # kendi anahtarını yaz

source_file_path = "genai_test_generator/src/math_utils.py"
output_dir = "genai_test_generator/tests"
os.makedirs(output_dir, exist_ok=True)

import ast

def extract_function_names_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        source = f.read()
    tree = ast.parse(source)
    return [node.name for node in tree.body if isinstance(node, ast.FunctionDef)]

def extract_function(source_code, function_name):
    lines = source_code.splitlines()
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith(f"def {function_name}("):
            start_index = i
            break
    if start_index is None:
        raise ValueError(f"{function_name} fonksiyonu bulunamadı.")
    fn_lines = [lines[start_index]]
    indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    for line in lines[start_index + 1:]:
        if len(line.strip()) == 0 or (len(line) - len(line.lstrip())) > indent:
            fn_lines.append(line)
        else:
            break
    return "\n".join(fn_lines)

def generate_test_code(function_code):
   #  prompt = f"""Aşağıdaki Python fonksiyonu için pytest kullanarak test fonksiyonları yaz.
   #  Sadece Python test kodu ver. Açıklama, yorum veya metin yazma:\n```python\n{function_code}\n```"""
    prompt = f"""
    Aşağıdaki Python fonksiyonu için pytest kullanarak test kodu üret.
    - Normal durumları test et
    - Edge-case senaryolarını da mutlaka dahil et (örneğin negatif sayılar, sıfır, yanlış veri tipi, boş giriş).
    Sadece Python test kodunu ver. Açıklama yazma.\n```python\n{function_code}\n```"""


    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

def clean_test_code(llm_output, function_name):
    # Kod bloklarını ayıkla (```python ... ```)
    code_blocks = re.findall(r"```(?:python)?(.*?)```", llm_output, re.DOTALL)
    cleaned = code_blocks[0].strip() if code_blocks else llm_output.strip()

    lines = cleaned.splitlines()
    new_lines = []
    import_found = False
    pytest_import_found = False

    for line in lines:
        stripped = line.strip()

        # pytest import varsa işaretle
        if stripped.startswith("import pytest"):
            pytest_import_found = True
            new_lines.append(line)
            continue

        # doğru import satırı varsa işaretle
        if (
            f"from genai_test_generator.src.math_utils import {function_name}" in stripped
            or re.match(rf"from\s+\S+\s+import\s+{function_name}", stripped)
        ):
            import_found = True
            continue  # tüm importları kaldırıp bizim ekleyeceğimiz için bunu atla

        # gereksiz diğer importları atla
        if stripped.startswith("import ") or stripped.startswith("from "):
            continue

        new_lines.append(line)

    # En başa pytest import'u yoksa ekle
    if not pytest_import_found:
        new_lines.insert(0, "import pytest")

    # pytest'ten sonra doğru math_utils import satırını ekle
    for i, line in enumerate(new_lines):
        if line.strip().startswith("import pytest"):
            new_lines.insert(i + 1, f"from genai_test_generator.src.math_utils import {function_name}")
            break
    else:
        new_lines.insert(0, f"from genai_test_generator.src.math_utils import {function_name}")

    return "\n".join(new_lines)





def get_test_count(test_path):
    result = subprocess.run(["pytest", test_path, "-q"], capture_output=True, text=True)
    output = result.stdout
    match = re.search(r"collected\s+(\d+)\s+items", output)
    if match:
        return int(match.group(1))
    return 0

def get_coverage():
    result = subprocess.run(["pytest", "--cov=genai_test_generator/src", "--cov-report=term"], capture_output=True, text=True)
    lines = result.stdout.splitlines()
    for line in lines:
        if "math_utils.py" in line:
            parts = line.split()
            try:
                return int(parts[3].replace("%", ""))
            except:
                pass
    return 0

source_code = open(source_file_path, "r", encoding="utf-8").read()

functions = extract_function_names_from_file(source_file_path)

data = {
    "Fonksiyon": [],
    "Test Tipi": [],
    "Kod Kapsamı (%)": [],
    "Test Sayısı": [],
    "Edge Case Durumu": [],
    "Üretim Süresi (sn)": [],
}

for fn in functions:
    try:
        fn_code = extract_function(source_code, fn)

        start_time = time.time()
        raw_output = generate_test_code(fn_code)
        elapsed = time.time() - start_time

        test_code = clean_test_code(raw_output, fn)

        test_path = os.path.join(output_dir, f"test_{fn}_llm.py")
        with open(test_path, "w", encoding="utf-8") as f_out:
            f_out.write(test_code)

        test_count = get_test_count(test_path)
        coverage = get_coverage()
        edge_case = "✅"  # İstersen daha detaylı kontrol ekleyebilirsin

        data["Fonksiyon"].append(fn)
        data["Test Tipi"].append("LLM")
        data["Kod Kapsamı (%)"].append(coverage if coverage is not None else 0)
        data["Test Sayısı"].append(test_count)
        data["Edge Case Durumu"].append(edge_case)
        data["Üretim Süresi (sn)"].append(round(elapsed, 2))

        print(f"✅ {fn} fonksiyonu için test dosyası oluşturuldu: {test_path}")
    except Exception as e:
        print(f"❌ {fn} fonksiyonu için test oluşturulurken hata oluştu: {e}")

df = pd.DataFrame(data)
print(df)


✅ add_numbers fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_add_numbers_llm.py
✅ divide_numbers fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_divide_numbers_llm.py
✅ compute_factorial fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_compute_factorial_llm.py
           Fonksiyon Test Tipi  Kod Kapsamı (%)  Test Sayısı Edge Case Durumu  \
0        add_numbers       LLM                0            0                ✅   
1     divide_numbers       LLM                0            0                ✅   
2  compute_factorial       LLM                0            0                ✅   

   Üretim Süresi (sn)  
0                8.38  
1                7.40  
2               11.98  


| Fonksiyon Adı             | Test Zorluğu | Edge Case Barındırır mı?   | Veri Tipi Çeşitliliği |
| ------------------------- | ------------ | -------------------------- | --------------------- |
| `summarize_statistics`    | Orta         | ✅ (boş liste, non-numeric) | Liste (float/int)     |
| `is_palindrome`           | Kolay        | ✅ (boş string, sayılar)    | String                |
| `check_password_strength` | Yüksek       | ✅ (regex, karma karakter)  | String                |


In [95]:
# genai_test_generator/src/math_utils.py
math_utils_code = """
def add_numbers(a: float, b: float) -> float:
    \"\"\"Adds two numeric values and returns the result.\"\"\"
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("Inputs must be numeric types.")
    return a + b

def divide_numbers(numerator: float, denominator: float) -> float:
    \"\"\"Divides the numerator by the denominator and returns the result.\"\"\"
    if not isinstance(numerator, (int, float)) or not isinstance(denominator, (int, float)):
        raise TypeError("Both numerator and denominator must be numeric.")
    if denominator == 0:
        raise ZeroDivisionError("Cannot divide by zero.")
    return numerator / denominator

def compute_factorial(n: int) -> int:
    \"\"\"Computes the factorial of a non-negative integer.\"\"\"
    if not isinstance(n, int):
        raise TypeError("Input must be an integer.")
    if n < 0:
        raise ValueError("Negative values are not allowed.")
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

def summarize_statistics(data: list) -> dict:
    \"\"\"    Returns basic statistics: mean, median, min, max of a list of numbers.    \"\"\"
    import statistics
    if not all(isinstance(x, (int, float)) for x in data):
        raise TypeError("All items must be numeric.")
    if not data:
        raise ValueError("Data list cannot be empty.")
    return {
        "mean": statistics.mean(data),
        "median": statistics.median(data),
        "min": min(data),
        "max": max(data)
    }

def is_palindrome(s: str) -> bool:
    \"\"\"
    Checks whether a string is a palindrome, ignoring case and spaces.
    \"\"\"
    if not isinstance(s, str):
        raise TypeError("Input must be a string.")
    cleaned = ''.join(c.lower() for c in s if c.isalnum())
    return cleaned == cleaned[::-1]

def check_password_strength(password: str) -> str:
    \"\"\"
    Returns strength level of a password: 'Weak', 'Medium', or 'Strong'.
    \"\"\"
    import re
    if not isinstance(password, str):
        raise TypeError("Password must be a string.")
    length = len(password)
    if length < 6:
        return "Weak"
    if re.search(r"[A-Z]", password) and re.search(r"\d", password) and re.search(r"[!@#$%^&*]", password):
        return "Strong"
    return "Medium"

"""

with open("genai_test_generator/src/math_utils.py", "w", encoding="utf-8") as f:
    f.write(math_utils_code)

print("✅ Gelişmiş math_utils.py dosyası oluşturuldu.")


✅ Gelişmiş math_utils.py dosyası oluşturuldu.


In [ ]:
import openai
import os
import re
import subprocess
import time
import pandas as pd

# OpenAI API anahtarını girin
openai.api_key = ""  # kendi anahtarını yaz

source_file_path = "genai_test_generator/src/math_utils.py"
output_dir = "genai_test_generator/tests"
os.makedirs(output_dir, exist_ok=True)

import ast

def extract_function_names_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        source = f.read()
    tree = ast.parse(source)
    return [node.name for node in tree.body if isinstance(node, ast.FunctionDef)]

def extract_function(source_code, function_name):
    lines = source_code.splitlines()
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith(f"def {function_name}("):
            start_index = i
            break
    if start_index is None:
        raise ValueError(f"{function_name} fonksiyonu bulunamadı.")
    fn_lines = [lines[start_index]]
    indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    for line in lines[start_index + 1:]:
        if len(line.strip()) == 0 or (len(line) - len(line.lstrip())) > indent:
            fn_lines.append(line)
        else:
            break
    return "\n".join(fn_lines)

def generate_test_code(function_code):
   #  prompt = f"""Aşağıdaki Python fonksiyonu için pytest kullanarak test fonksiyonları yaz.
   #  Sadece Python test kodu ver. Açıklama, yorum veya metin yazma:\n```python\n{function_code}\n```"""
    prompt = f"""
    Aşağıdaki Python fonksiyonu için pytest kullanarak test kodu üret.
    - Normal durumları test et
    - Edge-case senaryolarını da mutlaka dahil et (örneğin negatif sayılar, sıfır, yanlış veri tipi, boş giriş).
    Sadece Python test kodunu ver. Açıklama yazma.\n```python\n{function_code}\n```"""


    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

def clean_test_code(llm_output, function_name):
    # Kod bloklarını ayıkla (```python ... ``` arası)
    code_blocks = re.findall(r"```(?:python)?(.*?)```", llm_output, re.DOTALL)
    cleaned = code_blocks[0].strip() if code_blocks else llm_output.strip()

    lines = cleaned.splitlines()
    new_lines = []
    import_found = False
    pytest_import_found = False
    statistics_needed = False

    for line in lines:
        stripped = line.strip()

        # pytest varsa işaretle
        if stripped.startswith("import pytest"):
            pytest_import_found = True
            new_lines.append(line)
            continue

        # test fonksiyonları içinde mean/median kullanılmışsa istatistik modülünü işaretle
        if any(func in stripped for func in ["mean(", "median(", "statistics.mean", "statistics.median"]):
            statistics_needed = True

        # doğru import varsa işaretle
        if (
            f"from genai_test_generator.src.math_utils import {function_name}" in stripped
            or re.match(rf"from\s+\S+\s+import\s+{function_name}", stripped)
        ):
            import_found = True
            continue  # Zaten varsa, tekrar ekleme

        # Diğer tüm importları atla (sadece biz yöneteceğiz)
        if stripped.startswith("import ") or stripped.startswith("from "):
            continue

        new_lines.append(line)

    # En başa pytest import'u yoksa ekle
    if not pytest_import_found:
        new_lines.insert(0, "import pytest")

    # pytest'ten sonra math_utils import'u ekle
    for i, line in enumerate(new_lines):
        if line.strip().startswith("import pytest"):
            new_lines.insert(i + 1, f"from genai_test_generator.src.math_utils import {function_name}")
            break

    # İstatistik modülü gerekiyorsa ekle
    if statistics_needed:
        for i, line in enumerate(new_lines):
            if line.strip().startswith("import pytest"):
                new_lines.insert(i + 2, "from statistics import mean, median")
                break

    return "\n".join(new_lines)





def get_test_count(test_path):
    result = subprocess.run(["pytest", test_path, "-q"], capture_output=True, text=True)
    output = result.stdout
    match = re.search(r"collected\s+(\d+)\s+items", output)
    if match:
        return int(match.group(1))
    return 0

def get_coverage():
    result = subprocess.run(["pytest", "--cov=genai_test_generator/src", "--cov-report=term"], capture_output=True, text=True)
    lines = result.stdout.splitlines()
    for line in lines:
        if "math_utils.py" in line:
            parts = line.split()
            try:
                return int(parts[3].replace("%", ""))
            except:
                pass
    return 0

source_code = open(source_file_path, "r", encoding="utf-8").read()

functions = extract_function_names_from_file(source_file_path)

data = {
    "Fonksiyon": [],
    "Test Tipi": [],
    "Kod Kapsamı (%)": [],
    "Test Sayısı": [],
    "Edge Case Durumu": [],
    "Üretim Süresi (sn)": [],
}

for fn in functions:
    try:
        fn_code = extract_function(source_code, fn)

        start_time = time.time()
        raw_output = generate_test_code(fn_code)
        elapsed = time.time() - start_time

        test_code = clean_test_code(raw_output, fn)

        test_path = os.path.join(output_dir, f"test_{fn}_llm.py")
        with open(test_path, "w", encoding="utf-8") as f_out:
            f_out.write(test_code)

        test_count = get_test_count(test_path)
        coverage = get_coverage()
        edge_case = "✅"  # İstersen daha detaylı kontrol ekleyebilirsin

        data["Fonksiyon"].append(fn)
        data["Test Tipi"].append("LLM")
        data["Kod Kapsamı (%)"].append(coverage if coverage is not None else 0)
        data["Test Sayısı"].append(test_count)
        data["Edge Case Durumu"].append(edge_case)
        data["Üretim Süresi (sn)"].append(round(elapsed, 2))

        print(f"✅ {fn} fonksiyonu için test dosyası oluşturuldu: {test_path}")
    except Exception as e:
        print(f"❌ {fn} fonksiyonu için test oluşturulurken hata oluştu: {e}")

df = pd.DataFrame(data)
print(df)


✅ add_numbers fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_add_numbers_llm.py
✅ divide_numbers fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_divide_numbers_llm.py
✅ compute_factorial fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_compute_factorial_llm.py
✅ summarize_statistics fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_summarize_statistics_llm.py
✅ is_palindrome fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_is_palindrome_llm.py
✅ check_password_strength fonksiyonu için test dosyası oluşturuldu: genai_test_generator/tests/test_check_password_strength_llm.py
                 Fonksiyon Test Tipi  Kod Kapsamı (%)  Test Sayısı  \
0              add_numbers       LLM               98            0   
1           divide_numbers       LLM               98            0   
2        compute_factorial       LLM               98            0   
3     summariz